In [9]:
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
import re
import logging

# Helper Functions

In [10]:
def DatesDic2List(dates_dictionary):
    '''
    DatesDic2List takes a dictionary with the following format:
    { 'date_as_string': number of days as integer,...}
    returns a list with the range of dates of each item in the dictionary + the num of days 
    '''
    dates_list = []
    for date, date_range in dates_dictionary.items():
        dates_list.append(date)
        i = 1 
        sign = lambda a: (a>0) - (a<0)
        while(abs(date_range)>0):
            new_date = pd.to_datetime(date) + (sign(date_range) * timedelta(days=i))
            dates_list.append(new_date.strftime("%Y-%m-%d"))
            i = i+1
            date_range = date_range-(1*(sign (date_range)))
           
                    
    return list(set(dates_list))



date_list = DatesDic2List({'2020-03-27':-1,'2020-02-14':1})
date_list

['2020-03-27', '2020-02-14', '2020-02-15', '2020-03-26']

# Data Parsing Function

In [11]:
file_path = 'sample_data.xlsx'
file_path2 = 'latest/all_daily.csv'
date_range_dict = {'2020-01-22':3}
#date_range_dict = None


In [12]:
def InitLogger(file_path):
    # logger initiation
    
    # create logger 
    logger = logging.getLogger('sample')
    logger.setLevel(logging.INFO)
    logger.propagate = False
    
    # create file handler
    fh = logging.FileHandler(file_path,mode='w')
    fh.setLevel(logging.INFO)
    
    # create formatter and add it to the handlers
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)    
    logger.addHandler(fh)
    
    return(logger)

In [13]:
def ReadFile(file_path, _attr_date = 'date_ymd'):
    # Read File according to file extension
    if file_path.endswith(".xlsx"):
        xlsx = pd.ExcelFile(file_path)
        df = pd.read_excel(xlsx)
    elif file_path.endswith(".csv"):
        df = pd.read_csv(file_path, parse_dates=[_attr_date])
    return(df)

In [14]:
def ExtractRegionProvince(r_p_dic, df, _attr_region = 'Country_Region', _attr_province = 'Province_State'):
    # Region and Province Selection
    # r_p_dic = region_province_dictionary
    
    df_extract = df
    mask_prev =  [False]*len(df.index)
    if r_p_dic != {}:
        for region, provinces in r_p_dic.items():
            
            if (provinces == []) or (provinces == None) :
                mask = mask_prev | (df_extract[_attr_region] == region ) 
                mask_prev = mask
                continue
            for province in provinces:
                mask = mask_prev | ((df_extract[_attr_region] == region ) & (df_extract[_attr_province] == province)) 
                mask_prev = mask
        df_extract = df.loc[mask]
    else:
        df_extract = df
    return(df_extract)
    

In [15]:


def DataParser(file_path, date_range_dict, incubation_duration = 5, r_p_dic = {}, \
               attr_extract = None, _attr_date = 'date_ymd', _attr_region = 'Country_Region', \
               _attr_province = 'Province_State'):
    
    # init logger and log run parameters
    logger = InitLogger('dataparsing.log')
    logger.info('Incubation date: {}'.format(incubation_duration))
    
    # read data
    df = ReadFile(file_path, _attr_date)
    
    # extract regions and provinces from df
    df_extract = ExtractRegionProvince(r_p_dic, df, _attr_region, _attr_province)

    # DateParsing - create a mask to choose the desired dates (According to list and incubation period)
    df_final = pd.DataFrame() # creates a new dataframe that's empty
    if date_range_dict != None:
        date_list = DatesDic2List(date_range_dict)
        for date in date_list:
            # get the incubation period of the date
            curr_date_df = df_extract.loc[:,_attr_date: _attr_region]
            end_date = (pd.to_datetime(date)-timedelta(days=incubation_duration)) 
            mask = (df_extract[_attr_date] >= end_date) & (df_extract[_attr_date] <= date)
            incubation_df_extract = df_extract.loc[mask] # extracts the incubation period of the date
            att_df_final = incubation_df_extract.groupby([_attr_region],as_index=False).agg(attr_extract)
            att_df_final.insert(0,_attr_date,date,True)
            att_df_final.columns = att_df_final.columns.map('_'.join)
            # TO DO: join will add "_" to the end of columns titles that consist of one line, which is undesired
            #        make sure titles are back to normal
            mask_date =  [False]*len(df_extract.index)
            mask_date = mask_date | (df_extract[_attr_date] == date)
            for attr in attr_extract:
                df_date = df_extract.loc[mask_date]
                df_attr = df_date[[attr]]
                if len(df_attr.values.tolist())!=0:
                    att_df_final[attr] = pd.Series(df_attr.values.tolist()).str.get(0)
            if not att_df_final.empty:
                df_final = df_final.append(att_df_final, ignore_index = False) # ignoring index is optional
                df_final = df_final.sort_values(by=_attr_date+'_')
    else:
        df_final = df_extract.groupby(by=[_attr_region],as_index=False).agg(attr_extract) 
        df_final.insert(0,_attr_date,'ALL',True)
    
    df_final.reset_index(drop=True, inplace=True)
    
    return df_final

In [16]:
#df1 = DataParser(file_path2, None, _attr_date = 'date',r_p_dic={'Argentina':[],'Israel':[],'Mainland China':['Shaanxi']}, attr_extract={'Confirmed':['max','min']})


#date_range_dict = {'2020-01-22':80}
#df2 = DataParser(file_path2, date_range_dict, _attr_date = 'date',r_p_dic={'Argentina':[],'Israel':[],'Sweden':[],'Mainland China':['Shaanxi']}, attr_extract={'Confirmed':['mean']})
#df1
#df2

date_range_dict = {'2020-01-22':80}
df2 = DataParser(file_path2, date_range_dict, _attr_date = 'date',r_p_dic={'Argentina':[]}, attr_extract={'Confirmed':['mean']})
print("df2:")
print(df2)

# attr_extract={'confirmed_cases':['mean','min'],'mean_temp': ['max']} ,region=['Mainland China'], province = ['Anhui']

df2:
         date_ Country_Region_  Confirmed_mean  Confirmed
0   2020-03-03       Argentina        1.000000        1.0
1   2020-03-04       Argentina        1.000000        1.0
2   2020-03-05       Argentina        1.000000        1.0
3   2020-03-06       Argentina        1.250000        2.0
4   2020-03-07       Argentina        2.600000        8.0
5   2020-03-08       Argentina        4.166667       12.0
6   2020-03-09       Argentina        6.000000       12.0
7   2020-03-10       Argentina        8.666667       17.0
8   2020-03-11       Argentina       11.666667       19.0
9   2020-03-12       Argentina       14.500000       19.0
10  2020-03-13       Argentina       18.333333       31.0
11  2020-03-14       Argentina       22.000000       34.0
12  2020-03-15       Argentina       27.500000       45.0
13  2020-03-16       Argentina       34.000000       56.0
14  2020-03-17       Argentina       42.166667       68.0
15  2020-03-18       Argentina       52.166667       79.0
16  2020-